In [3]:
# Add execute permissions to binaries
!chmod +x dssp
!chmod +x DAlphaBall.gcc

# Verify permissions
!ls -l dssp
!ls -l DAlphaBall.gcc

-rwxr-xr-x@ 1 satishgaurav  staff  877904 Jan 28 10:21 dssp
-rwxr-xr-x@ 1 satishgaurav  staff  345824 Jan 28 10:21 DAlphaBall.gcc


In [4]:
####################################
################ BioPython functions
####################################
### Import dependencies
import os
import math
import numpy as np
from collections import defaultdict
from scipy.spatial import cKDTree
from Bio import BiopythonWarning
from Bio.PDB import PDBParser, DSSP, Selection, Polypeptide, PDBIO, Select, Chain, Superimposer
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.PDB.Selection import unfold_entities
from Bio.PDB.Polypeptide import is_aa

# analyze sequence composition of design
def validate_design_sequence(sequence, num_clashes, advanced_settings):
    note_array = []

    # Check if protein contains clashes after relaxation
    if num_clashes > 0:
        note_array.append('Relaxed structure contains clashes.')

    # Check if the sequence contains disallowed amino acids
    if advanced_settings["omit_AAs"]:
        restricted_AAs = advanced_settings["omit_AAs"].split(',')
        for restricted_AA in restricted_AAs:
            if restricted_AA in sequence:
                note_array.append('Contains: '+restricted_AA+'!')

    # Analyze the protein
    analysis = ProteinAnalysis(sequence)

    # Calculate the reduced extinction coefficient per 1% solution
    extinction_coefficient_reduced = analysis.molar_extinction_coefficient()[0]
    molecular_weight = round(analysis.molecular_weight() / 1000, 2)
    extinction_coefficient_reduced_1 = round(extinction_coefficient_reduced / molecular_weight * 0.01, 2)

    # Check if the absorption is high enough
    if extinction_coefficient_reduced_1 <= 2:
        note_array.append(f'Absorption value is {extinction_coefficient_reduced_1}, consider adding tryptophane to design.')

    # Join the notes into a single string
    notes = ' '.join(note_array)

    return notes

# temporary function, calculate RMSD of input PDB and trajectory target
def target_pdb_rmsd(trajectory_pdb, starting_pdb, chain_ids_string):
    # Parse the PDB files
    parser = PDBParser(QUIET=True)
    structure_trajectory = parser.get_structure('trajectory', trajectory_pdb)
    structure_starting = parser.get_structure('starting', starting_pdb)
    
    # Extract chain A from trajectory_pdb
    chain_trajectory = structure_trajectory[0]['A']
    
    # Extract the specified chains from starting_pdb
    chain_ids = chain_ids_string.split(',')
    residues_starting = []
    for chain_id in chain_ids:
        chain_id = chain_id.strip()
        chain = structure_starting[0][chain_id]
        for residue in chain:
            if is_aa(residue, standard=True):
                residues_starting.append(residue)
    
    # Extract residues from chain A in trajectory_pdb
    residues_trajectory = [residue for residue in chain_trajectory if is_aa(residue, standard=True)]
    
    # Ensure that both structures have the same number of residues
    min_length = min(len(residues_starting), len(residues_trajectory))
    residues_starting = residues_starting[:min_length]
    residues_trajectory = residues_trajectory[:min_length]
    
    # Collect CA atoms from the two sets of residues
    atoms_starting = [residue['CA'] for residue in residues_starting if 'CA' in residue]
    atoms_trajectory = [residue['CA'] for residue in residues_trajectory if 'CA' in residue]
    
    # Calculate RMSD using structural alignment
    sup = Superimposer()
    sup.set_atoms(atoms_starting, atoms_trajectory)
    rmsd = sup.rms
    
    return round(rmsd, 2)

# detect C alpha clashes for deformed trajectories
def calculate_clash_score(pdb_file, threshold=2.4, only_ca=False):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)

    atoms = []
    atom_info = []  # Detailed atom info for debugging and processing

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.element == 'H':  # Skip hydrogen atoms
                        continue
                    if only_ca and atom.get_name() != 'CA':
                        continue
                    atoms.append(atom.coord)
                    atom_info.append((chain.id, residue.id[1], atom.get_name(), atom.coord))

    tree = cKDTree(atoms)
    pairs = tree.query_pairs(threshold)

    valid_pairs = set()
    for (i, j) in pairs:
        chain_i, res_i, name_i, coord_i = atom_info[i]
        chain_j, res_j, name_j, coord_j = atom_info[j]

        # Exclude clashes within the same residue
        if chain_i == chain_j and res_i == res_j:
            continue

        # Exclude directly sequential residues in the same chain for all atoms
        if chain_i == chain_j and abs(res_i - res_j) == 1:
            continue

        # If calculating sidechain clashes, only consider clashes between different chains
        if not only_ca and chain_i == chain_j:
            continue

        valid_pairs.add((i, j))

    return len(valid_pairs)

three_to_one_map = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

# identify interacting residues at the binder interface
def hotspot_residues(trajectory_pdb, binder_chain="B", atom_distance_cutoff=4.0):
    # Parse the PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", trajectory_pdb)

    # Get the specified chain
    binder_atoms = Selection.unfold_entities(structure[0][binder_chain], 'A')
    binder_coords = np.array([atom.coord for atom in binder_atoms])

    # Get atoms and coords for the target chain
    target_atoms = Selection.unfold_entities(structure[0]['A'], 'A')
    target_coords = np.array([atom.coord for atom in target_atoms])

    # Build KD trees for both chains
    binder_tree = cKDTree(binder_coords)
    target_tree = cKDTree(target_coords)

    # Prepare to collect interacting residues
    interacting_residues = {}

    # Query the tree for pairs of atoms within the distance cutoff
    pairs = binder_tree.query_ball_tree(target_tree, atom_distance_cutoff)

    # Process each binder atom's interactions
    for binder_idx, close_indices in enumerate(pairs):
        binder_residue = binder_atoms[binder_idx].get_parent()
        binder_resname = binder_residue.get_resname()

        # Convert three-letter code to single-letter code using the manual dictionary
        if binder_resname in three_to_one_map:
            aa_single_letter = three_to_one_map[binder_resname]
            for close_idx in close_indices:
                target_residue = target_atoms[close_idx].get_parent()
                interacting_residues[binder_residue.id[1]] = aa_single_letter

    return interacting_residues

# calculate secondary structure percentage of design
def calc_ss_percentage(pdb_file, advanced_settings, chain_id="B", atom_distance_cutoff=4.0):
    # Parse the structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]  # Consider only the first model in the structure

    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')

    # Prepare to count residues
    ss_counts = defaultdict(int)
    ss_interface_counts = defaultdict(int)
    plddts_interface = []
    plddts_ss = []

    # Get chain and interacting residues once
    chain = model[chain_id]
    interacting_residues = set(hotspot_residues(pdb_file, chain_id, atom_distance_cutoff).keys())

    for residue in chain:
        residue_id = residue.id[1]
        if (chain_id, residue_id) in dssp:
            ss = dssp[(chain_id, residue_id)][2]  # Get the secondary structure
            ss_type = 'loop'
            if ss in ['H', 'G', 'I']:
                ss_type = 'helix'
            elif ss == 'E':
                ss_type = 'sheet'

            ss_counts[ss_type] += 1

            if ss_type != 'loop':
                # calculate secondary structure normalised pLDDT
                avg_plddt_ss = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_ss.append(avg_plddt_ss)

            if residue_id in interacting_residues:
                ss_interface_counts[ss_type] += 1

                # calculate interface pLDDT
                avg_plddt_residue = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_interface.append(avg_plddt_residue)

    # Calculate percentages
    total_residues = sum(ss_counts.values())
    total_interface_residues = sum(ss_interface_counts.values())

    percentages = calculate_percentages(total_residues, ss_counts['helix'], ss_counts['sheet'])
    interface_percentages = calculate_percentages(total_interface_residues, ss_interface_counts['helix'], ss_interface_counts['sheet'])

    i_plddt = round(sum(plddts_interface) / len(plddts_interface) / 100, 2) if plddts_interface else 0
    ss_plddt = round(sum(plddts_ss) / len(plddts_ss) / 100, 2) if plddts_ss else 0

    return (*percentages, *interface_percentages, i_plddt, ss_plddt)

def calculate_percentages(total, helix, sheet):
    helix_percentage = round((helix / total) * 100,2) if total > 0 else 0
    sheet_percentage = round((sheet / total) * 100,2) if total > 0 else 0
    loop_percentage = round(((total - helix - sheet) / total) * 100,2) if total > 0 else 0

    return helix_percentage, sheet_percentage, loop_percentage

In [5]:
advanced_settings = {
    "dssp_path": "/usr/local/bin/mkdssp",
    "omit_AAs": "C",
}
pdb_file = "./../out/bindcraft/2501230908/Accepted/7z14_l75_s376417_mpnn7_model2.pdb"
calc_ss_percentage(pdb_file=pdb_file, advanced_settings=advanced_settings)

(42.67, 30.67, 26.67, 23.81, 47.62, 28.57, 0.93, 0.96)

In [6]:
import pandas as pd

AA_PROPERTIES = {
        "ALA": "nonpolar",
        "ARG": "positive",
        "ASN": "polar",
        "ASP": "negative",
        "CYS": "polar",
        "GLN": "polar",
        "GLU": "negative",
        "GLY": "nonpolar",
        "HIS": "positive",
        "ILE": "nonpolar",
        "LEU": "nonpolar",
        "LYS": "positive",
        "MET": "nonpolar",
        "PHE": "nonpolar",
        "PRO": "nonpolar",
        "SER": "polar",
        "THR": "polar",
        "TRP": "nonpolar",
        "TYR": "polar",
        "VAL": "nonpolar",
        "SEC": "polar"
    }

def create_structure_df(pdb_file, advanced_settings, chain_id=None, atom_distance_cutoff=4.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]
    
    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')
    
    # Prepare data for DataFrame
    data = []
    
    # Process either specific chain or all chains
    chains = [model[chain_id]] if chain_id else model.get_chains()
    
    for chain in chains:
        current_chain_id = chain.id
        # Get interacting residues if chain_id is specified
        interacting_residues = set()
        if current_chain_id:
            interacting_residues = set(hotspot_residues(pdb_file, current_chain_id, atom_distance_cutoff).keys())
        
        for residue in chain:
            residue_id = residue.id[1]
            res_name = residue.get_resname()
            
            if (current_chain_id, residue_id) in dssp:
                dssp_data = dssp[(current_chain_id, residue_id)]
                ss = dssp_data[2]
                acc = dssp_data[3]
                phi = dssp_data[4]
                psi = dssp_data[5]
                
                ss_type = 'loop'
                if ss in ['H', 'G', 'I']:
                    ss_type = 'helix'
                elif ss == 'E':
                    ss_type = 'sheet'
                    
                avg_plddt = sum(atom.bfactor for atom in residue) / len(residue) / 100
                
                data.append({
                    'chain_id': current_chain_id,
                    'residue_id': residue_id,
                    'residue_name': res_name,
                    'property': AA_PROPERTIES.get(res_name, 'unknown'),
                    'ss_type': ss_type,
                    'ss_code': ss,
                    'accessibility': acc,
                    'phi': phi,
                    'psi': psi,
                    'plddt': round(avg_plddt, 2),
                    'is_interface': residue_id in interacting_residues if chain_id else None
                })
    
    return pd.DataFrame(data)


# Example usage:
pdb_file = "./../out/bindcraft/2501230908/Accepted/7z14_l75_s376417_mpnn7_model2.pdb"
df = create_structure_df(pdb_file, advanced_settings)
df.head()

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,MET,nonpolar,loop,-,0.292553,360.0,144.7,0.91,None
1,A,2,ILE,nonpolar,sheet,E,0.230769,-102.1,120.5,0.96,None
2,A,3,CYS,polar,sheet,E,0.007407,-115.4,146.4,0.97,None
3,A,4,TYR,polar,sheet,E,0.189189,-76.1,140.5,0.98,None
4,A,5,ASN,polar,loop,-,0.318471,-130.0,20.1,0.98,None


In [7]:
df.tail(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
125,B,66,PHE,nonpolar,loop,-,0.005076,-102.5,140.4,0.93,None
126,B,67,ALA,nonpolar,sheet,E,0.103774,-151.0,153.8,0.95,None
127,B,68,MET,nonpolar,sheet,E,0.000000,-137.5,138.4,0.97,None
128,B,69,ASP,negative,sheet,E,0.110429,-103.4,117.7,0.98,None
129,B,70,ILE,nonpolar,sheet,E,0.017751,-105.8,114.8,0.98,None
130,B,71,ILE,nonpolar,sheet,E,0.076923,-119.2,124.0,0.97,None
131,B,72,VAL,nonpolar,sheet,E,0.007042,-119.7,121.1,0.97,None
132,B,73,ARG,positive,sheet,E,0.560484,-131.3,150.6,0.95,None
133,B,74,PRO,nonpolar,sheet,E,0.647059,-69.1,151.0,0.93,None
134,B,75,LYS,positive,loop,-,0.917073,-80.4,360.0,0.89,None


In [8]:
df.head(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,MET,nonpolar,loop,-,0.292553,360.0,144.7,0.91,None
1,A,2,ILE,nonpolar,sheet,E,0.230769,-102.1,120.5,0.96,None
2,A,3,CYS,polar,sheet,E,0.007407,-115.4,146.4,0.97,None
3,A,4,TYR,polar,sheet,E,0.189189,-76.1,140.5,0.98,None
4,A,5,ASN,polar,loop,-,0.318471,-130.0,20.1,0.98,None
5,A,6,GLN,polar,loop,-,0.015152,-83.9,145.1,0.96,None
6,A,7,GLN,polar,loop,-,0.595960,-98.9,146.0,0.93,None
7,A,8,SER,polar,loop,T,0.484615,49.5,-120.2,0.93,None
8,A,9,SER,polar,loop,T,0.861538,-105.5,20.5,0.93,None
9,A,10,GLN,polar,loop,P,0.520202,-60.8,151.1,0.95,None


In [9]:
df.to_csv('secondary_structure_information.csv', index=False)

In [10]:
df_target = df.head(60)

In [11]:
df_target.to_csv('7z14_secondary_structure_information.csv', index=False)

In [12]:
df_target[df_target['property'] == 'nonpolar']

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,MET,nonpolar,loop,-,0.292553,360.0,144.7,0.91,None
1,A,2,ILE,nonpolar,sheet,E,0.230769,-102.1,120.5,0.96,None
10,A,11,PRO,nonpolar,loop,P,0.764706,-58.4,138.7,0.96,None
11,A,12,PRO,nonpolar,loop,P,0.705882,-53.8,136.1,0.96,None
26,A,27,TRP,nonpolar,sheet,E,0.264317,-151.3,165.9,0.96,None
31,A,32,GLY,nonpolar,loop,-,0.357143,163.9,-166.8,0.95,None
33,A,34,ILE,nonpolar,sheet,E,0.295858,-120.5,128.8,0.97,None
34,A,35,ILE,nonpolar,sheet,E,0.230769,-120.2,123.2,0.98,None
37,A,38,GLY,nonpolar,sheet,E,0.047619,175.1,-178.0,0.98,None
39,A,40,GLY,nonpolar,loop,S,0.154762,93.1,-177.2,0.94,None


In [14]:
df_target[df_target['property'] == 'nonpolar'].sort_values('accessibility', ascending=False)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
10,A,11,PRO,nonpolar,loop,P,0.764706,-58.4,138.7,0.96,None
11,A,12,PRO,nonpolar,loop,P,0.705882,-53.8,136.1,0.96,None
46,A,47,GLY,nonpolar,loop,T,0.630952,96.1,-10.2,0.94,None
41,A,42,PRO,nonpolar,loop,-,0.419118,-71.9,156.8,0.95,None
31,A,32,GLY,nonpolar,loop,-,0.357143,163.9,-166.8,0.95,None
45,A,46,PRO,nonpolar,loop,T,0.308824,-53.6,142.1,0.96,None
33,A,34,ILE,nonpolar,sheet,E,0.295858,-120.5,128.8,0.97,None
0,A,1,MET,nonpolar,loop,-,0.292553,360.0,144.7,0.91,None
26,A,27,TRP,nonpolar,sheet,E,0.264317,-151.3,165.9,0.96,None
1,A,2,ILE,nonpolar,sheet,E,0.230769,-102.1,120.5,0.96,None


In [17]:
df_target.sort_values('accessibility', ascending=False)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
29,A,30,HIS,positive,loop,S,0.951087,-78.9,-4.7,0.95,None
17,A,18,SER,polar,loop,S,0.876923,-82.3,-11.2,0.94,None
15,A,16,THR,polar,sheet,E,0.873239,-79.9,128.0,0.98,None
8,A,9,SER,polar,loop,T,0.861538,-105.5,20.5,0.93,None
44,A,45,LYS,positive,loop,-,0.790244,-76.3,155.6,0.94,None
30,A,31,ARG,positive,loop,S,0.770161,-106.2,-39.2,0.94,None
10,A,11,PRO,nonpolar,loop,P,0.764706,-58.4,138.7,0.96,None
11,A,12,PRO,nonpolar,loop,P,0.705882,-53.8,136.1,0.96,None
27,A,28,ARG,positive,sheet,E,0.701613,-122.2,148.1,0.96,None
42,A,43,LYS,positive,loop,-,0.668293,-67.0,149.4,0.95,None
